In [1]:
model_path = "v1.pt"

In [2]:
import torch
from torch import nn
# 신경망 정의
class Res18Net(nn.Module):
    def __init__(self):
        super(Res18Net, self).__init__()
        self.model = resnet18(weights=ResNet18_Weights.DEFAULT)
        self.model.conv1 = nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, 5)

    def forward(self, x):
        x = self.model(x)
        return x

In [3]:
import torch
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
model = torch.load(model_path, map_location=device)

In [4]:
# 데이터셋 처리 함수 정의
def path2tensor(path, label):
    data_x = []
    data_y = []
    
    files = os.listdir(path)
    for f in files:
        f = os.path.join(path, f)
        img = cv2.imread(f).astype(np.float32) / 255
        img = np.transpose(img, (2, 0, 1))
        data_x.append(img)

        y = [0 for i in range(5)]
        y[label] = 1
        data_y.append(y)

    return (data_x, data_y)

In [5]:
import os, cv2

img_path = "E:\\AI_data\\test_set"

str2label = { "bird" : 0,
              "car" : 1,
              "cat" : 2,
              "dog" : 3,
              "fish" : 4 }

test_x = []
test_y = []

for c in ["cat", "fish"]:
    result = path2tensor(os.path.join(img_path, c), str2label[c])
    test_x += result[0]
    test_y += result[1]

test_x = np.array(test_x)
test_y = np.array(test_y)

test_x = torch.tensor(test_x, dtype=torch.float32)
test_y = torch.tensor(test_y, dtype=torch.float32)

In [17]:
# 모델 평가하기
target_data = []
sig_data = []
pred_data = []
target_n = []

model.eval()

with torch.no_grad():
    inputs = test_x.to(device)
    outputs = model(inputs)
    sig = torch.sigmoid(outputs)
    
    sig_data.extend(sig.cpu().numpy())
    target_data.extend(test_y.cpu().numpy())

    target_n.extend(torch.argmax(test_y, dim=1).cpu().numpy())
    _, predicted = torch.max(outputs, 1)
    pred_data.extend(predicted.cpu().numpy())

In [18]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# 예측값과 실제값을 NumPy 배열로 변환
pred = np.array(pred_data)
target = np.array(target_n)

# 정확도 계산
accuracy = accuracy_score(target, pred)

# 정밀도 계산
precision = precision_score(target, pred, average='weighted')

# 재현율 계산
recall = recall_score(target, pred, average='weighted', zero_division=1)

# F1 점수 계산
f1Score = f1_score(target, pred, average='weighted')

In [19]:
print(f'정확도: {accuracy}')
print(f'정밀도: {precision}')
print(f'재현율: {recall}')
print(f'F1 점수: {f1Score}')

정확도: 0.575
정밀도: 1.0
재현율: 0.575
F1 점수: 0.7272727272727273


In [20]:
diff_index = []
for i in range(len(target_data)):
    if not np.array_equal(target_n[i], pred_data[i]):
        diff_index.append(i)

print(f"{len(diff_index)}개의 데이터 예측에 실패했습니다.")
print()
print("예측 실패 : ", diff_index)

17개의 데이터 예측에 실패했습니다.

예측 실패 :  [0, 2, 3, 8, 11, 14, 15, 16, 17, 18, 25, 26, 27, 33, 36, 37, 38]


In [68]:
import matplotlib.pyplot as plt
import numpy as np
import copy
from IPython.display import display
import ipywidgets as widgets

def display_images_with_probs(images, pred_percs, target_percs):
    
    # 이미지와 확률을 나란히 표시할 위젯 생성
    output = widgets.Output()
    with output:
        for img, pred_perc, target_perc in zip(images, pred_percs, target_percs):
            plt.figure(figsize=(5, 5))
            img = np.transpose(np.squeeze(img), (1, 2, 0))  # CHW -> HWC 변환
            img = cv2.cvtColor(img.numpy(), cv2.COLOR_BGR2RGB)
            plt.imshow(img)
            plt.axis('off')  # 축 제거
            plt.title(f"pred: {pred_perc}\ntarget: {target_perc}")
            plt.show()
    
    display(output)

images = [test_x[answer] for answer in diff_index]
pred_percs = [list(copy.deepcopy(sig_data[answer])) for answer in diff_index]
target_percs = [copy.deepcopy(target_data[answer]) for answer in diff_index]

for i, d in enumerate(pred_percs):
    for ii, p in enumerate(d):
        pred_percs[i][ii] = round(p, 3)

display_images_with_probs(images, pred_percs, target_percs)

Output()